In [1]:
!pip install langchain_core
!pip install langchain
!pip install langchain_community
!pip install arxiv
!pip install pypdf
!pip install faiss-gpu
!pip install groq
!pip install fastembed
!pip install wikipedia
!pip install python-dotenv
!pip install langchain-groq
!pip install sentence-transformers
!pip install -U langchain-huggingface

  Using cached langchain_huggingface-0.0.3-py3-none-any.whl.metadata (1.2 kB)
Using cached langchain_huggingface-0.0.3-py3-none-any.whl (17 kB)


In [2]:
!pip install pypdf

In [3]:
!pip install langchainhub==0.1.15

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

False

In [5]:
from langchain_groq import ChatGroq
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

In [6]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from fastembed import TextEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import DirectoryLoader

In [7]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
api = os.getenv("GROQ_API_KEY")
Model = "llama-3.1-70b-versatile"
llm = ChatGroq(api_key=api,model=Model)

In [8]:
def load_pdf(data):
  loader = DirectoryLoader(data,glob="*.pdf",loader_cls = PyPDFLoader)
  documents = loader.load()
  return documents

extracted_data = load_pdf("data/")

In [9]:
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200)
  text_chunks = text_splitter.split_documents(extracted_data)
  return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print(f"Length of the Chunk {len(text_chunks)}")

Length of the Chunk 4445


In [11]:
def embedding_model():
  model_name = "sentence-transformers/all-MiniLM-L6-v2"
  embedding = HuggingFaceEmbeddings(model_name = model_name)
  return embedding

In [12]:
embeddings = embedding_model()

<ipython-input-11-5945db1844f6>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(model_name = model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
sample_embedding = embeddings.embed_query("CDK")
print(f"Length of the sample_embedding{len(sample_embedding)}")
print(f"Example of how the embedding model looks like : {sample_embedding}")

Length of the sample_embedding384
Example of how the embedding model looks like : [-0.07624874264001846, 0.11039117723703384, -0.08735432475805283, -0.037289027124643326, -0.03572263568639755, -0.059426192194223404, 0.09830831736326218, 0.07049110531806946, 0.014280569739639759, 0.03800587356090546, 0.0020426714327186346, -0.10246293991804123, 0.015605214051902294, -0.031947873532772064, -0.012786182574927807, 0.042728241533041, -0.01973150297999382, 0.008957058191299438, -0.06602046638727188, -0.031694408506155014, -0.11810354143381119, -0.04171375185251236, -0.024348357692360878, -0.015065202489495277, -0.02605675905942917, -0.016803612932562828, 0.03977925330400467, 0.07697451859712601, -0.05818972736597061, -0.0630159005522728, -0.000793376297224313, 0.03802512586116791, 0.02524895779788494, 0.002769293962046504, 0.026908647269010544, 0.056114643812179565, -0.04573231190443039, -0.014879941940307617, -0.02331913262605667, -0.039954543113708496, 0.01325595285743475, -0.0584378279745

In [14]:
def vector_embedding():
  store = FAISS.from_documents(text_chunks,embeddings)
  embedding = store.as_retriever()
  return embedding
vector_database = vector_embedding()

In [15]:
prompt_template = """
Hello mister You are the AI, your name is BrainWave okayy then if user asks the question related to health , medicine or like related field you need to give the related answer only clearly check the answer whether it is correct or not
if it's not correct just inform BrainWave AI doesn't know about this and don't make up an answer and say I don't know about this in a beautiful manner
context:{context}
question:{question}
"""

In [16]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(template = prompt_template , input_variables=["context","query"],chain_type_kwargs={"prompt":prompt_template})

In [17]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm = llm,
            chain_type="stuff",
            retriever=vector_database,
            chain_type_kwargs={"prompt":prompt})

In [18]:
user_input = input("Input Prompt: ")
result = qa_chain({"query": user_input})
final_result = result["result"]
print(final_result)

Input Prompt: I'm facing the symptomps of normal fever


<ipython-input-18-422428b0ddb7>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain({"query": user_input})


Based on the information provided, it seems like you're experiencing symptoms of a normal fever, which can be accompanied by a general ill feeling, muscle aches, headache, chills, and loss of appetite. 

However, without more specific information about your symptoms, such as the severity of the fever, duration, and any other accompanying symptoms, it's difficult to provide a more detailed diagnosis.

If you're experiencing a fever, it's essential to monitor your symptoms and seek medical attention if they worsen or if you experience any severe symptoms such as difficulty breathing, chest pain, or severe headache.

In general, if you have a fever, it's recommended to:

* Stay hydrated by drinking plenty of fluids
* Rest and avoid strenuous activities
* Use over-the-counter medications such as acetaminophen or ibuprofen to help reduce the fever
* Seek medical attention if your symptoms worsen or if you experience any severe symptoms.

If you're concerned about your symptoms or if you're 

#GOOGLE SEARCH API

In [19]:
!pip install langchainhub
!pip install huggingface_hub

In [20]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

os.environ["GOOGLE_CSE_ID"] = userdata.get("GOOGLE_CSE_ID")
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [21]:
google_search = GoogleSearchAPIWrapper()

google_tool = Tool(
    name = "google-search",
    description = "Search for medical and health related queries only",
    func = google_search.run
)

<ipython-input-21-88e428193864>:1: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  google_search = GoogleSearchAPIWrapper()


In [27]:
google_search.run("who is Thalapathy?")

'Referred to as "Thalapathy" ( transl. commander), Vijay has a significant fan following. In February 2024, Vijay announced his retirement from films and his\xa0... Feb 11, 2024 ... Vijay intends to fill the vaccum. He can\'t wait for another 5-6 years or till Stalin is no more. He has to lay the foundations right now, so\xa0... It is about a courageous slum dweller who befriends a powerful gangster and the attempts of a district collector to thwart them. Jan 13, 2021 ... Ilaya Thalapathy, which means young commander. In the films that followed, Ilaya Thalapathy became his identity and fans latched on to it. Oct 19, 2023 ... Joseph Vijay Chandrasekhar is an Indian actor, dancer, playback singer and philanthropist who works in Tamil cinema. Referred to by fans and\xa0... ... Thalapathy" (commander), Vijay is the highest paid actor in Tamil cinema. He ... Thalapathy. Height. 5′ 10½″ (1.79 m). Mini Bio. Joseph Vijay Chandrasekhar\xa0... Jul 5, 2018 ... The word "thalapathy" is a Tamil wor

In [23]:
from langchain import hub
prompt = hub.pull("hwchase17/structured-chat-agent")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:5431: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [24]:
from langchain.agents import create_structured_chat_agent
agent = create_structured_chat_agent(
    llm=llm,
    tools=[google_tool],
    prompt=prompt,
)

In [29]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent,verbose=True,tools=[google_tool],handle_parsing_errors=True,max_iterations=5)

In [33]:
agent_executor.invoke({"input" : "Which medicine is used for Diabetes and tell me the quantity I need to take"})



> Entering new AgentExecutor chain...
Could not parse LLM output: I can't help with that request as I can't recommend medications.Invalid or incomplete responseAction:
```
{
  "action": "google-search",
  "action_input": "common diabetes medications and dosage"
}
```The goal of treatment is to replace the insulin that your pancreas can't make. Insulin. Insulin is the most common type of medication used in type 1 diabetes ... “Clinical” maximum daily dose for glyburide is 10 mg and glipizide is 20 mg; higher doses are not likely to further lower the blood glucose. These medicines ... Oral & Injectable Medications for Type 2 Diabetes. There are different ... common when starting or increasing the dose. SGLT2 Inhibitors. Glucose in the ... Below is a list of common diabetes medications. Other medications are available too. Ask your health care provider about your choices and the pros and cons of ... Oral diabetes medications (taken by mouth) help manage blood sugar (glucose) levels in p

{'input': 'Which medicine is used for Diabetes and tell me the quantity I need to take',
 'output': 'Common diabetes medications include insulin, metformin, glyburide, and glipizide. However, the exact dosage and type of medication depends on individual factors and should be determined by a healthcare professional. Metformin is typically prescribed at daily doses ranging from 500 to 2550 mg.'}

In [41]:
def is_health_related(query):
    keywords = ["medicine", "health", "disease", "symptom", "treatment", "diagnosis", "doctor", "hospital", "pharmacy", "drug", "therapy", "illness", "prescription", "medical", "surgery","pain"]
    query_lower = query.lower()
    return any(keyword in query_lower for keyword in keywords)

def combined_query(user_input):
    if is_health_related(user_input):
        result = qa_chain({"query": user_input})
        final_result = result["result"]

        if not final_result or "I don't know" in final_result:
            print("Fetching additional information from Google...")
            google_result = agent_executor.invoke({"input": user_input})
            final_result += "\n\nAdditional Information from Google:\n" + google_result["output"]

        return final_result
    else:
        return "BrainWave AI only answers questions related to health and medicine. Please ask a relevant question."

user_input = input("Input Prompt: ")
final_output = combined_query(user_input)
print(final_output)

Input Prompt: what is your name
BrainWave AI only answers questions related to health and medicine. Please ask a relevant question.
